In [1]:
!pip install torch_geometric


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
# Imports
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
# PyG
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

# Vostri moduli
from src.loadData    import GraphDataset
from src.models      import GNN
from src.transforms import EdgeDropout, NodeDropout, Compose
from src.losses import (
    GeneralizedCELoss,
    SymmetricCELoss,
    estimate_transition_matrix,
    ForwardCorrectionLoss,
    BootstrappingLoss
)
from src.pretraining import GraphCLTrainer, add_zeros
from src.divide_mix_def import DivideMixTrainer

# Fissa seed
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)



In [4]:
def add_zeros(data):
    if not hasattr(data, 'x') or data.x is None:
        data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [5]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    
    model = model.to(device)
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    print(f"Train loss/acc: {total_loss / len(data_loader):.4f}/{correct / total:.4f}")
    return total_loss / len(data_loader),  correct / total

In [6]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)

            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())

    
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
        print(f"Test loss/acc {total_loss / len(data_loader):.4f} / {correct / total:.4f}")
    return predictions

In [7]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd()
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))

    os.makedirs(submission_folder, exist_ok=True)

    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")

    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })

    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [8]:
def plot_training_progress(train_losses, train_accuracies, save_plot, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    if(save_plot):
        # Save plots in the current directory
        os.makedirs(output_dir, exist_ok=True)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "training_progress.png"))
        plt.close()

In [9]:
# Hyper-parameters
device                =  "cuda" if torch.cuda.is_available() else "cpu"
# Modello
gnn_type              = 'gin'   
num_layer             = 3
emb_dim               = 300
drop_ratio            = 0.6
pooling               = "attention"

edge_p  = 0.2   # frazione di bordi da droppare
node_p  = 0.2
# Ottimizzazione / Scheduler
lr                    = 0.01
epochs                = 100
weight_decay          = 5e-4
t_max                 = 50
eta_min               = 1e-5
num_classes           = 6
batch_size            = 32 
patience              = 12
residual              = True
transforms = Compose([
    add_zeros,
])



# Co-Teaching

In [10]:
def create_gnn_model(gnn_type, num_classes, num_layer, emb_dim, drop_ratio, pooling, device, residual):
    kwargs = {
        'gnn_type': gnn_type.replace("-virtual", ""),
        'num_class': num_classes,
        'num_layer': num_layer,
        'emb_dim': emb_dim,
        'drop_ratio': drop_ratio,
        'virtual_node': "virtual" in gnn_type,
        'residual': residual,
        'graph_pooling': pooling
        }

    model = GNN(**kwargs).to(device)
    return model


def create_optimizer_and_scheduler(model, lr=lr, weight_decay=weight_decay, t_max=t_max, eta_min=eta_min):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=t_max, eta_min=eta_min)
    return optimizer, scheduler



In [11]:
def select_small_loss(loss, forget_rate):
    """Select indices of samples with small losses (based on the forget rate)."""
    if loss.ndim == 0 or len(loss) == 0:
        return torch.tensor([], dtype=torch.long)
    remember_rate = 1 - forget_rate
    num_remember = max(1, int(remember_rate * len(loss)))
    return torch.topk(-loss, k=num_remember, largest=True).indices

def validate_metrics(model, val_loader, device):
    """Validate model performance on clean validation set using accuracy and F1 score."""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.to(device)
            outputs = model(batch)
            preds = outputs.argmax(dim=1).cpu()
            labels = batch.y.cpu()
            
            all_preds.append(preds)
            all_labels.append(labels)
    
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()
    
    val_f1 = f1_score(all_labels, all_preds, average='macro')
    val_acc = accuracy_score(all_labels, all_preds)
    return val_acc, val_f1

def train_with_coteaching(
    model1, model2,
    train_dataset,
    val_dataset,
    criterion,
    optimizer1, optimizer2,
    scheduler1, scheduler2,
    device,
    num_epochs=100,
    forget_rate=0.2,
    patience=10,
    batch_size=32
):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    reference_model = model1
    best_val_f1 = 0
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(num_epochs):
        model1.train()
        model2.train()

        total_loss1, total_loss2 = 0, 0
        total_train_samples = 0
        all_train_preds = []
        all_train_labels = []
        correct_train = 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            batch = batch.to(device)
            labels = batch.y

            outputs1 = model1(batch)
            outputs2 = model2(batch)

            loss1 = criterion(outputs1, labels)
            loss2 = criterion(outputs2, labels)

            idx1 = select_small_loss(loss2.detach(), forget_rate)
            idx2 = select_small_loss(loss1.detach(), forget_rate)
            
            if len(idx1) == 0 or len(idx2) == 0:
                continue 

            final_loss1 = torch.mean(loss1[idx1])
            final_loss2 = torch.mean(loss2[idx2])

            total_loss1 += final_loss1.item() * len(idx1)
            total_loss2 += final_loss2.item() * len(idx2)
            total_train_samples += len(idx1)

            with torch.no_grad():
                preds = outputs1[idx2].argmax(dim=1)
                correct_train += (preds == labels[idx2]).sum().item()
                all_train_preds.append(preds.cpu())
                all_train_labels.append(labels[idx2].cpu())
                
            optimizer1.zero_grad()
            final_loss1.backward()
            optimizer1.step()

            optimizer2.zero_grad()
            final_loss2.backward()
            optimizer2.step()

        scheduler1.step()
        scheduler2.step()
        
        # Compute train metrics
        avg_train_loss = (total_loss1 + total_loss2) / (2 * total_train_samples) if total_train_samples > 0 else 0
        train_acc = correct_train / total_train_samples if total_train_samples > 0 else 0
        if all_train_preds and all_train_labels:
            train_preds = torch.cat(all_train_preds).numpy()
            train_labels = torch.cat(all_train_labels).numpy()
            train_f1 = f1_score(train_labels, train_preds, average='macro')
        else:
            train_f1 = 0.0

        # Validation metrics
        val_acc, val_f1 = validate_metrics(reference_model, val_loader, device)
        
        print(f"Epoch {epoch+1}: "
              f"Train Loss={avg_train_loss:.4f}, "
              f"Train Acc={train_acc:.4f}, "
              f"Train F1={train_f1:.4f}, "
              f"Val Acc={val_acc:.4f}, "
              f"Val F1={val_f1:.4f}, "
              f"(Best Val F1={best_val_f1:.4f})")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_model_state = deepcopy(reference_model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break
        
    if best_model_state is not None:
        reference_model.load_state_dict(best_model_state)
    
    return reference_model

In [12]:
def split_dataset(dataset: GraphDataset, val_ratio=0.1, seed=42):
    labels = torch.tensor([data.y.item() for data in dataset])
    indices = list(range(len(dataset)))

    train_idx, val_idx = train_test_split(
        indices,
        test_size=val_ratio,
        stratify=labels,
        random_state=seed
    )

    train_subset = []
    val_subset = []

    for new_idx, original_idx in enumerate(train_idx):
        data = dataset[original_idx]
        data.idx = new_idx  # Normalize index
        train_subset.append(data)

    for new_idx, original_idx in enumerate(val_idx):
        data = dataset[original_idx]
        data.idx = new_idx  # Normalize index
        val_subset.append(data)

    return train_subset, val_subset


In [13]:
train_datasets_path = [ "datasets/B/train.json.gz"]

In [14]:
import gc

for ds in train_datasets_path:

    model_kwargs = {"gnn_type": gnn_type, "num_classes":num_classes, "num_layer": num_layer, "emb_dim": emb_dim, "drop_ratio": drop_ratio, "device":device, "residual": residual, "pooling": pooling }
    model1 = create_gnn_model(**model_kwargs)
    model2 = create_gnn_model(**model_kwargs)

    optimizer1, scheduler1 = create_optimizer_and_scheduler(model1, lr=lr, weight_decay=weight_decay)
    optimizer2, scheduler2 = create_optimizer_and_scheduler(model2, lr=lr, weight_decay=weight_decay)

    print("Loading dataset")
    full_dataset = GraphDataset(ds, transform=transforms)
    criterion = nn.CrossEntropyLoss(reduction="none")
    
    train_set, test_set = split_dataset(full_dataset, 0.2)
    best_model = train_with_coteaching(model1=model1,
                                       model2=model2,
                                       train_dataset=train_set,
                                       val_dataset=test_set,
                                       optimizer1=optimizer1,
                                       optimizer2=optimizer2,
                                       scheduler1=scheduler1,
                                       scheduler2=scheduler2,
                                       device=device,
                                       num_epochs=epochs,
                                       patience = patience,
                                       criterion=criterion)

    del model1, model2, full_dataset, optimizer1, scheduler1, optimizer2, scheduler2, train_set, test_set
    gc.collect()

    test_loader = DataLoader(GraphDataset(ds.replace("train", "test"), transform=transforms), batch_size=32)
    predictions = evaluate(test_loader, best_model, device, False)
    save_predictions(predictions=predictions, test_path=ds)

    del test_loader, predictions 
    gc.collect()

Loading dataset


Epoch 1/100: 100%|██████████| 140/140 [00:08<00:00, 16.76it/s]


Epoch 1: Train Loss=1.7649, Train Acc=0.2877, Train F1=0.2096, Val Acc=0.2161, Val F1=0.1001, (Best Val F1=0.0000)


Epoch 2/100: 100%|██████████| 140/140 [00:08<00:00, 17.04it/s]


Epoch 2: Train Loss=1.5550, Train Acc=0.2977, Train F1=0.1859, Val Acc=0.2679, Val F1=0.0936, (Best Val F1=0.1001)


...

Epoch 57: Train Loss=0.8838, Train Acc=0.7214, Train F1=0.6627, Val Acc=0.5643, Val F1=0.5054, (Best Val F1=0.5104)


Epoch 58/100: 100%|██████████| 140/140 [00:08<00:00, 17.28it/s]


Epoch 58: Train Loss=0.8915, Train Acc=0.7123, Train F1=0.6513, Val Acc=0.5652, Val F1=0.5057, (Best Val F1=0.5104)
Early stopping triggered!


Iterating eval graphs: 100%|██████████| 49/49 [00:03<00:00, 13.63batch/s]


Predictions saved to /home/flaviolinux/uni/deep_learning/hackaton/submission/testset_B.csv
